In [1]:
import pandas as pd
import numpy as np

In [8]:
df_a = pd.read_csv("web/data/brand_values.csv")
df_a

,name,value,year,lastValue,rank
0,Apple,214480.0,2018.0,211447.4,1
1,Apple,211447.4,2017.9,208414.8,1
2,Apple,208414.8,2017.8,205382.2,1
3,Apple,205382.2,2017.7,202349.6,1
4,Apple,202349.6,2017.6,199317.0,1
...,...,...,...,...,...
18095,Zara,NaN,2004.5,NaN,100
18096,Zara,NaN,2004.6,NaN,100
18097,Zara,NaN,2004.7,NaN,100
18098,Zara,NaN,2004.8,NaN,100


In [67]:
df_b = pd.read_csv("web/data/covid-data.csv")
df_b["date"] = pd.to_datetime(df_b["date"])
df_b = df_b[["date", "location", "total_cases", "new_cases"]]
df_b["lastValue"] = df_b["total_cases"] - df_b["new_cases"]
df_b = df_b.rename(columns={"total_cases" : "value", "location" : "name"})
df_b = df_b.fillna(0)
df_b = df_b.sort_values(["date", "value"], ascending=[True, False])
df_b["date"] = df_b["date"].apply(lambda x: round(x.isocalendar()[0] + x.isocalendar()[1]/52, 1))
df_b = df_b.groupby(["date", "name"]).mean().reset_index()
df_b = df_b.rename(columns={"date":"year"})
cont = ["Europe", "Asia", "North America", "Africa", "Oceania", "World", "South America", "European Union", "International"]
df_b = df_b[~df_b["name"].isin(cont)]
df_b["rank"] = df_b.groupby("year").rank(method="max", ascending=False)["value"].astype(int)
df_b = df_b.replace(0, np.nan)
df_b = df_b.iloc[5:,:]
df_b = df_b.sort_values(["name", "year"], ascending=[True, False])
df_b["lastValue"] = 0
df_tot = pd.DataFrame()
for pays in df_b["name"].unique():
    df_temp = df_b[df_b["name"]==pays].copy()
    df_temp["lastValue"] = df_temp["value"].shift(periods=-1).fillna(0)
    df_tot = pd.concat([df_tot,df_temp])
df_tot = df_tot.sort_values(["name", "year"], ascending=[True, False])
# df_tot.loc[df_tot["year"] == 2020.1, 'lastValue'] = df_tot.loc[df_tot["year"] == 2020.1, 'value']
for i,name in enumerate(df_tot.loc[df_tot["year"] == 2020.1].sort_values("value", ascending= False)['name'].unique()):
    df_tot = df_tot.append({'year':2020.0,'name':name,'value':0,'new_cases':0, 'lastValue':0, 'rank':i+1}, ignore_index=True)
df_tot

,year,name,value,new_cases,lastValue,rank
0,2021.4,Afghanistan,65637.045455,454.363636,58709.571429,105
1,2021.3,Afghanistan,58709.571429,147.600000,56027.428571,105
2,2021.2,Afghanistan,56027.428571,25.523810,55086.742857,103
3,2021.1,Afghanistan,55086.742857,46.285714,51056.761905,97
4,2021.0,Afghanistan,51056.761905,159.000000,44225.600000,90
...,...,...,...,...,...,...
2607,2020.0,Morocco,0.000000,0.000000,0.000000,37
2608,2020.0,Peru,0.000000,0.000000,0.000000,38
2609,2020.0,Portugal,0.000000,0.000000,0.000000,39
2610,2020.0,Slovenia,0.000000,0.000000,0.000000,40


In [68]:
df_tot.to_csv("web/data/covid_2.csv")

In [34]:
df_tot[df_tot["name"]=="Italy"]

,year,name,value,new_cases,lastValue,rank
2577,2021.4,Italy,4.175916e+06,4845.954545,3.911293e+06,8
2352,2021.3,Italy,3.911293e+06,12655.600000,3.233513e+06,8
2130,2021.2,Italy,3.233513e+06,20452.809524,2.598246e+06,7
1912,2021.1,Italy,2.598246e+06,12227.685714,2.060867e+06,8
1701,2021.0,Italy,2.060867e+06,15533.309524,1.276291e+06,7
1499,2020.9,Italy,1.276291e+06,29129.800000,4.062205e+05,9
1302,2020.8,Italy,4.062205e+05,9789.976190,2.740320e+05,15
1107,2020.7,Italy,2.740320e+05,1264.028571,2.475757e+05,19
912,2020.6,Italy,2.475757e+05,310.114286,2.394026e+05,15
717,2020.5,Italy,2.394026e+05,230.371429,2.264652e+05,9


In [63]:
df_tot.loc[df_tot["year"] == 2020.1]

,year,name,value,new_cases,lastValue,rank
124,2020.1,Australia,12.000000,0.681818,0.0,9
229,2020.1,Belgium,1.000000,0.076923,0.0,29
391,2020.1,Cambodia,1.000000,0.047619,0.0,29
418,2020.1,Canada,4.909091,0.318182,0.0,17
489,2020.1,China,25953.884615,2688.769231,0.0,1
597,2020.1,Cyprus,NaN,NaN,0.0,43
637,2020.1,Denmark,NaN,NaN,0.0,43
703,2020.1,Egypt,1.000000,0.333333,0.0,29
756,2020.1,Estonia,NaN,NaN,0.0,43
803,2020.1,Fiji,NaN,NaN,0.0,43


In [61]:
for i,name in enumerate(df_tot.loc[df_tot["year"] == 2020.1].sort_values("value", ascending= False)['name'].unique()):
    df_tot = df_tot.append({'year':2020.0,'name':name,'value':0,'new_cases':0, 'lastValue':0, 'rank':i+1}, ignore_index=True)

In [62]:
df_tot

,year,name,value,new_cases,lastValue,rank
0,2021.4,Afghanistan,65637.045455,454.363636,58709.571429,105
1,2021.3,Afghanistan,58709.571429,147.600000,56027.428571,105
2,2021.2,Afghanistan,56027.428571,25.523810,55086.742857,103
3,2021.1,Afghanistan,55086.742857,46.285714,51056.761905,97
4,2021.0,Afghanistan,51056.761905,159.000000,44225.600000,90
...,...,...,...,...,...,...
2622,2020.0,Morocco,0.000000,0.000000,0.000000,38
2623,2020.0,Peru,0.000000,0.000000,0.000000,39
2624,2020.0,Portugal,0.000000,0.000000,0.000000,40
2625,2020.0,Slovenia,0.000000,0.000000,0.000000,41


In [64]:
df_tot.loc[df_tot["year"] == 2020.0]

,year,name,value,new_cases,lastValue,rank
2585,2020.0,China,0.0,0.0,0.0,1
2586,2020.0,International,0.0,0.0,0.0,2
2587,2020.0,Singapore,0.0,0.0,0.0,3
2588,2020.0,Hong Kong,0.0,0.0,0.0,4
2589,2020.0,Japan,0.0,0.0,0.0,5
2590,2020.0,Thailand,0.0,0.0,0.0,6
2591,2020.0,South Korea,0.0,0.0,0.0,7
2592,2020.0,Australia,0.0,0.0,0.0,8
2593,2020.0,Malaysia,0.0,0.0,0.0,9
2594,2020.0,United Kingdom,0.0,0.0,0.0,10


In [10]:
df = pd.read_csv("web/data/covid-data.csv")
df["date"] = pd.to_datetime(df["date"])
df = df[["date", "location", "total_cases", "new_cases"]]
df["lastValue"] = df["total_cases"] - df["new_cases"]
df = df.rename(columns={"total_cases" : "value", "location" : "name"})
df = df.fillna(0)
df = df.sort_values(["date", "value"], ascending=[True, False])
df = df.groupby(["name", pd.Grouper(key='date', freq="W-MON")]).sum("value").reset_index()
df["lastValue"] = df["value"].shift().fillna(0)
df = df.drop("new_cases", axis=1)
df = df.rename(columns={"date":"year"})
df["rank"] = 0
df = df[["name", "value", "year", "lastValue", "rank"]]#.to_csv("covid_1.csv")
df["year"] = df["year"].dt.year + df["year"].dt.week/52
df["year"] = df["year"].round(1)
df = df.groupby(["name", "year"]).max().reset_index()
df["lastValue"] = df["value"].shift().fillna(0)
cont = ["Europe", "Asia", "North America", "Africa", "Oceania", "World", "South America", "European Union"]
df = df[~df["name"].isin(cont)]
df["rank"] = df.groupby("year").rank(method="max", ascending=False)["value"].astype(int)

<ipython-input-10-acba7a4e34d8>:14: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df["year"] = df["year"].dt.year + df["year"].dt.week/52


In [42]:
df

,name,year,value,lastValue,rank
0,Afghanistan,2020.2,203.0,0.0,101
1,Afghanistan,2020.3,9537.0,203.0,75
2,Afghanistan,2020.4,96483.0,9537.0,45
3,Afghanistan,2020.5,226442.0,96483.0,44
4,Afghanistan,2020.6,258809.0,226442.0,54
...,...,...,...,...,...
2684,Zimbabwe,2021.0,137606.0,67837.0,111
2685,Zimbabwe,2021.1,245052.0,137606.0,106
2686,Zimbabwe,2021.2,257528.0,245052.0,112
2687,Zimbabwe,2021.3,267681.0,257528.0,113


In [13]:
df[df["name"] == 'France']

,name,year,value,lastValue,rank
873,France,2020.1,57.0,645178.0,15
874,France,2020.2,91911.0,57.0,7
875,France,2020.3,1122956.0,91911.0,4
876,France,2020.4,1320257.0,1122956.0,7
877,France,2020.5,1442613.0,1320257.0,13
878,France,2020.6,1654264.0,1442613.0,18
879,France,2020.7,2823286.0,1654264.0,12
880,France,2020.8,7588473.0,2823286.0,5
881,France,2020.9,15732193.0,7588473.0,4
882,France,2021.0,19601090.0,15732193.0,6


In [8]:
df.to_csv("web/data/covid_1.csv")

In [5]:
#df.groupby(["name", pd.Grouper(key='date', freq='W-MON')]).sum("value")

dates = df["date"].unique()
df["rank"] = 0
for day in dates:
    for i in range(len(df[df["date"] == day])):
        df[df["date"] == day].iloc[i]["rank"] = i+1

In [6]:
df_a

,name,value,year,lastValue,rank
0,Apple,214480.0,2018.0,211447.4,1
1,Apple,211447.4,2017.9,208414.8,1
2,Apple,208414.8,2017.8,205382.2,1
3,Apple,205382.2,2017.7,202349.6,1
4,Apple,202349.6,2017.6,199317.0,1
...,...,...,...,...,...
18095,Zara,NaN,2004.5,NaN,100
18096,Zara,NaN,2004.6,NaN,100
18097,Zara,NaN,2004.7,NaN,100
18098,Zara,NaN,2004.8,NaN,100


In [7]:
df

,name,year,value,lastValue,rank
0,Afghanistan,2020.2,203.0,0.0,101
1,Afghanistan,2020.3,9537.0,203.0,75
2,Afghanistan,2020.4,96483.0,9537.0,45
3,Afghanistan,2020.5,226442.0,96483.0,44
4,Afghanistan,2020.6,258809.0,226442.0,54
...,...,...,...,...,...
2684,Zimbabwe,2021.0,137606.0,67837.0,111
2685,Zimbabwe,2021.1,245052.0,137606.0,106
2686,Zimbabwe,2021.2,257528.0,245052.0,112
2687,Zimbabwe,2021.3,267681.0,257528.0,113
